In [3]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import cmath
import numpy as np
import matplotlib.pyplot as plt
from sympy import primerange
import random
import networkx as nx
from tqdm.notebook import tqdm
import pickle
from scipy.stats import wasserstein_distance as w1_dist
import scipy as sp
from multiprocessing import Pool
import time
from scipy.sparse import eye, csr_matrix
import spectral_density as spec
import ot
from scipy.stats import wasserstein_distance

In [6]:
''' Copied from Apoorv's implementation with a few modifications to make matrices scipy sparse '''

def eig_normAdj(G):
    #Returns normalized adjacency eigenvalues
    return np.ones(nx.number_of_nodes(G)) - nx.normalized_laplacian_spectrum(G).real

# def get_normAdj(G):
#     # Returns the normalized adjacency matrix
#     return np.eye(G.number_of_nodes()) - nx.normalized_laplacian_matrix(G).toarray()

def get_normAdj(G):
    # Calculate the normalized Laplacian matrix (already in sparse format)
    norm_laplacian = nx.normalized_laplacian_matrix(G)
    # Identity matrix in sparse format, directly created as sparse for efficiency
    I = eye(G.number_of_nodes(), format='csr')
    # Subtract the normalized Laplacian matrix from the identity matrix
    sparse_norm_adj_matrix = I - norm_laplacian
    return sparse_norm_adj_matrix

def find_vtx_index(vtx_u, vtx_v, adj_list_G):
    # Finds the index of vtx_v in the adjacency list of vtx_u
    for idx in range(len(adj_list_G[vtx_u])):
        if adj_list_G[vtx_u][idx] == vtx_v:
            return idx
        
def replacement_prod(G,l):
    adj_list_G = {}
    # Create a dictionary of neighbour lists
    for vtx in G.nodes():
        neighbour_list = list(G.neighbors(vtx))
        adj_list_G[vtx] = neighbour_list
    # G_rep: Initialize the replacement product graph
    G_rep = nx.Graph()
    # Create the nodes of the replacement product graph 
    # Indexed by the original vertex and the index of the neighbour
    for vtx in G.nodes():
        for j in range(l):
            G_rep.add_node((vtx,j))
    # Create the edges of the replacement product graph from the original graph G
    for vtx in G.nodes():
        for j in range(l):
            j_th_neighbour = adj_list_G[vtx][j]
            idx_of_vtx = find_vtx_index(j_th_neighbour, vtx, adj_list_G)
            G_rep.add_edge((vtx,j),(j_th_neighbour,idx_of_vtx))
    # Create the edges to the cycle of the replacement product graph
    for vtx in G.nodes():
        for j in range(l):
            if j == l-1:
                k = 0
                G_rep.add_edge((vtx,j),(vtx,k))
            else:
                G_rep.add_edge((vtx,j),(vtx,j+1))
    return G_rep


def compute_tridiag(A, k, m=3):
    # run Lanczos and get recurrence coefficients
    αβ_list = []
    d = A.shape[0]
    for _ in range(m):
        # generate random vector
        v = np.random.randn(d)
        v /= np.linalg.norm(v)
        # run Lanczos and append coefficients
        αβ_list.append(spec.lanczos(A,v,k,reorth=False))
    return αβ_list

def compute_w(f_x, g_x, x): 
    # Normalize histograms
    f_x /= np.sum(f_x)
    g_x /= np.sum(g_x)
    # Define the ground metric
    M = np.abs(x[:, np.newaxis] - x[np.newaxis, :])
    wasserstein_dist = ot.emd2(f_x, g_x, M)
    return wasserstein_dist

def compute_distance():
    l = []
    for i in range(5,15,2):
        l.append(i)
    w1_lst = []
    for i in tqdm(l, desc='Iterating over ls'):
        
        print("Starting l = " , str(i))
        
        # Determine size of graph and target error (set to be like 1/l^3)
        n = int(np.ceil(np.log(i) * 2 ** i))
        if n % 2 == 1:
            n = n + 1
        error = 1/(2*i**3)
        
        # set N in KPM according to the target error (set accoridng to Tyler's paper)
        N = int(np.ceil(36/error))
        
        print("\t generating graphs... ")
        G2 = nx.random_regular_graph(2*i,n)
        G2_r = replacement_prod(G2, 2*i)
        AG2_r = get_normAdj(G2_r)
        
        G1 = nx.random_regular_graph(i,n)
        G1_r = replacement_prod(G1, i)
        AG1_r = get_normAdj(G1_r)
        
        # number of LDOS to average (set according to Tyler's suggestion)
        m = 10
        
        print("Matrix shape is", AG1_r.shape)
        print("\t running first lanczos... ")
        start_time = time.time()
        αβ_list1 = compute_tridiag(AG1_r, int(np.ceil(N/2)), m=m)
        end_time = time.time()
        print(
            "\t\t completed in ", 
            str(end_time-start_time), 
            " seconds, i.e. ", 
            str((end_time-start_time)/int(np.ceil(N/2))), 
            " seconds per iteration."
        )
        
#         print("\t computing first density... ")
#         σ1 = spec.get_arcsin_density(-1,1, N)
#         ρ_KPM1 = spec.KPM(αβ_list1,σ1)
        
#         print("\t running second lanczos... ")
#         start_time = time.time()
#         αβ_list2 = compute_tridiag(AG2_r, int(np.ceil(N/2)), m=m)
#         end_time = time.time()
#         print(
#             "\t\t completed in ", 
#             str(end_time-start_time), 
#             " seconds, i.e. ", 
#             str((end_time-start_time)/int(np.ceil(N/2))), 
#             " seconds per iteration."
#         )
#         print("\t computing second density... ")
#         σ2 = spec.get_arcsin_density(-1,1, N)
#         ρ_KPM2 = spec.KPM(αβ_list2,σ2)
        
        
#         print("\t computing w1 distance... ")
#         x = np.linspace(-1,1, 20000)
#         y1 = ρ_KPM1(x,damping='Jackson')
#         y2 = ρ_KPM2(x,damping='Jackson')
        
#         w1 = compute_w(y2, y1, x)
#         w1_lst.append(w1)
        
#         print("\t Completed l = ", str(i), " and W1 distance is ", str(w1))
    return w1_lst

In [7]:
w1_lst = compute_distance()

Iterating over ls:   0%|          | 0/5 [00:00<?, ?it/s]

Starting l =  5
	 generating graphs... 
Matrix shape is (260, 260)
	 running first lanczos... 
		 completed in  0.8709361553192139  seconds, i.e.  0.0001935413678487142  seconds per iteration.
Starting l =  7
	 generating graphs... 
Matrix shape is (1750, 1750)
	 running first lanczos... 
		 completed in  4.317597150802612  seconds, i.e.  0.00034965963320396923  seconds per iteration.
Starting l =  9
	 generating graphs... 
Matrix shape is (10134, 10134)
	 running first lanczos... 
		 completed in  34.213019132614136  seconds, i.e.  0.001303601414845271  seconds per iteration.
Starting l =  11
	 generating graphs... 
Matrix shape is (54032, 54032)
	 running first lanczos... 
		 completed in  292.2389569282532  seconds, i.e.  0.006098984826117647  seconds per iteration.
Starting l =  13
	 generating graphs... 
Matrix shape is (273182, 273182)
	 running first lanczos... 
		 completed in  2942.208472967148  seconds, i.e.  0.03719982391350766  seconds per iteration.
